## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["token_transfers"],
    range_spec="@20241030:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-17 18:00:22 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=5921
2024-12-17 18:00:22 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=5921
2024-12-17 18:00:22 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=5921
2024-12-17 18:00:22 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=5921
2024-12-17 18:00:22 [info     ] prepared 1 input batches.      filename=bydate.py lineno=95 process=5921


## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory


execute_model_in_memory(
    duckdb_client=duckdb_client,
    model="token_transfers",
    data_reader=batch,
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
#
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-17 18:00:22 [info     ] Executing model function...    filename=testutils.py lineno=220 process=5921
2024-12-17 18:00:22 [info     ] duckdb dataset='ingestion/traces_v1' using 23/23 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/traces_v1/chain=op/dt=2024-10-30/000127322000.parquet filename=reader.py lineno=72 process=5921
2024-12-17 18:00:23 [info     ] registered view: 'ingestion_traces_v1' using 23 parquet paths filename=client.py lineno=53 process=5921
2024-12-17 18:00:23 [info     ] duckdb dataset='ingestion/logs_v1' using 23/23 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=op/dt=2024-10-30/000127322000.parquet filename=reader.py lineno=72 process=5921
2024-12-17 18:00:24 [info     ] registered view: 'ingestion_logs_v1' using 23 parquet paths filename=client.py lineno=53 process=5921
2024-12-17 18:00:24 [info     ] duckdb dataset='ingestion/transactions_v1' using 23/23 parquet paths, first path is gs://oplabs-tools-dat

┌───────────────────────────┐
│           name            │
│          varchar          │
├───────────────────────────┤
│ ingestion_logs_v1         │
│ ingestion_traces_v1       │
│ ingestion_transactions_v1 │
│ native_transfers          │
│ native_transfers_v1       │
└───────────────────────────┘

## Verify model results

In [3]:
duckdb_client.sql("SELECT *, value_64/1e18 AS value_native FROM native_transfers_v1 LIMIT 10")

┌─────────┬──────────┬─────────┬────────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────────┬────────────────────┬────────────┬───────────┬─────────────────┬────────────────┬───────────┬───────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────────┬──────────────────────┐
│ network │ chain_id │  chain  │     dt     │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │           transfer_from_address            │            transfer_to_address             │ trace_method_id │      value_64      │   value_lossless   │ trace_type

### You can also convert the results to dataframes to inspect them in more familiar ways

In [4]:
duckdb_client.sql("SELECT * FROM native_transfers_v1 LIMIT 10").pl().head()

network,chain_id,chain,dt,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,transfer_from_address,transfer_to_address,trace_method_id,value_64,value_lossless,trace_type,call_type,trace_gas_limit,trace_gas_used,subtraces,trace_address,tx_from_address,tx_to_address,tx_method_id
str,i32,str,date,u32,i64,str,str,i64,str,str,str,i64,str,str,str,i64,i64,i64,str,str,str,str
"""mainnet""",10,"""op""",2024-10-30,1730250601,127325912,"""0x091e27e20a77c42779efb7c7af67…","""0xaf039d9965ee1ed1a047d06fe80f…",7,"""0x1632ed73c1a866fe2c21e6fd0f49…","""0xe4edb277e41dc89ab076a1f049f4…","""0x""",269375236255299057,"""269375236255299057""","""call""","""call""",21000,21000,0,"""""","""0x1632ed73c1a866fe2c21e6fd0f49…","""0xe4edb277e41dc89ab076a1f049f4…","""0x"""
"""mainnet""",10,"""op""",2024-10-30,1730250603,127325913,"""0x6a338444de812763401d4065f575…","""0xf958157473bb2d066d8225ec07ec…",34,"""0x4200000000000000000000000000…","""0x1c6858664ccfacc2dcc8175f663c…","""0x""",57459274561,"""57459274561""","""call""","""call""",2300,75,0,"""26,6,3,0""","""0x3c9779d3ec32ed4985a3e634c44d…","""0x346c0b53faea3eaa013aebe3019b…","""0x00001a39"""
"""mainnet""",10,"""op""",2024-10-30,1730250603,127325913,"""0x6a338444de812763401d4065f575…","""0xf958157473bb2d066d8225ec07ec…",34,"""0x1c6858664ccfacc2dcc8175f663c…","""0xabe58b50367fb4cea7d19c41c228…","""0x""",57459274561,"""57459274561""","""call""","""call""",2300,0,0,"""26,6,4""","""0x3c9779d3ec32ed4985a3e634c44d…","""0x346c0b53faea3eaa013aebe3019b…","""0x00001a39"""
"""mainnet""",10,"""op""",2024-10-30,1730250603,127325913,"""0x6a338444de812763401d4065f575…","""0x995ff15d8712d4b727f388dd9814…",40,"""0xc7b7bccba82df66e9c219751fcf3…","""0xa2a786ff9148f7c88ee93372db8c…","""0xba677db7""",2418622215047,"""2418622215047""","""call""","""call""",486358,433983,13,"""""","""0xc7b7bccba82df66e9c219751fcf3…","""0xa2a786ff9148f7c88ee93372db8c…","""0xba677db7"""
"""mainnet""",10,"""op""",2024-10-30,1730250603,127325913,"""0x6a338444de812763401d4065f575…","""0x995ff15d8712d4b727f388dd9814…",40,"""0xa2a786ff9148f7c88ee93372db8c…","""0x94a365ca808029af8db18257ecd2…","""0x""",2418622215047,"""2418622215047""","""call""","""call""",377551,0,0,"""3""","""0xc7b7bccba82df66e9c219751fcf3…","""0xa2a786ff9148f7c88ee93372db8c…","""0xba677db7"""
